# Goal: Implement two fundamental parsing algorithms—Cocke-Kasami-Younger (CKY) algorithm and Earley’s algorithm—for parsing sentences based on a context-free grammar (CFG).
> **Notes to help**
>
> `Grammar` denotes the syntactical rules for conversation in natural language.
>
> It is also the set of rules that generate strings
>
> `language` the set of all strings that can be generated from a grammar
>
> **Context Free Grammar**
>
> G = V X R S
>
> such that:
>
> 1. V -- a finite set of variables(*non-terminal words*)
>
> 2. X -- a finite set of terminal symbols
>
> 3. R -- a finite set of rules
>
> 4. S -- the start symbol, `a distinct element of V`
>
> **Note** V and X are disjoint sets.
>
> **Membership Problem** -- grammar G generates a language `L(G)`. is a given string s a member of `L(G)`
>

> **Chomsky Normal Form**
>
> Grammar G is in `CNF` form if:
>
> - **A** --> BC *two non-terminal symbols on the RHS*
>
> - **A** --> a *one terminal symbol on the RHS*
> - **S** --> nullstring
>
> **Cocke-Younger-Kasami**
>
> Uses `dynamic programming`
>
> - The solution to a problem *[i, j]* can be constructed from a solution to subproblem *[i, k]* and solution to subproblem *[k, j]*
>
> - It needs the Grammar G to be in CNF
>
> For a string of length N,
> 1. construct a table T of size N x N.
>
> 2. Each cell in the table T[i, j] is the set of all constituents that can produce the substring spanning from position i to j.
> 3. fill the table with the solutions to the subproblems encountered in the bottom-up parsing process. `Therefore, cells will be filled from left to right and bottom to top`.

||1|2|3|4|5|
|:---:|:---:|:---:|:---:|:---:|:---:|
|1| [1, 1]| [1, 2] | [1, 3]| [1, 4]| [1, 5]|
|2| |[2, 2] | [2, 3]| [2, 4]| [2, 5]|
|3| | |[3, 3]| [3, 4]| [3, 5]|
|4| | | | [4, 4]| [4, 5]|
|5| | | | | [5, 5]|
>
> $$ A \in T[i, j] if\ and\ only\ if\\
B \in T[i, k],\\
C \in T[k, j],\\
and A → BC is\ a\ rule\ of G$$
>



## Task 1: Use the following pseudocode structure to implement CKY:

1. Convert the given CFG into Chomsky Normal Form (CNF).
2. Create a 2D table to represent substrings of the input sentence.
3. Iterate over substrings and fill in the table based on grammar rules.
Return whether the input sentence is valid or not according to the CFG

In [2]:
pdf_sent = "The cat sat on the mat" # based on the pdf
pptx_sent = "The cat chased the mouse" #based on the powerpoint

In [3]:
# packages
import nltk
from nltk import CFG

In [4]:
# create the grammar

grammar = CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    VP -> V NP | V PP
    PP -> P NP
    Det -> 'the'
    N -> 'cat' | 'mat' | 'mouse'
    V -> 'sat' | 'chased'
    P -> 'on'
""")

In [5]:
# create a parser

parser = nltk.ChartParser(grammar)

In [6]:
# tokenize the sentence
pdf_sent_tokens = pdf_sent.lower().split()
pptx_sent_tokens = pptx_sent.lower().split()

In [7]:
pdf_sent_tokens

['the', 'cat', 'sat', 'on', 'the', 'mat']

In [8]:
# print the trees from parsing
print("PDF Sentence Parsing:")
for tree in parser.parse(pdf_sent_tokens):
    print(tree)
    tree.pretty_print()

PDF Sentence Parsing:
(S
  (NP (Det the) (N cat))
  (VP (V sat) (PP (P on) (NP (Det the) (N mat)))))
             S                     
      _______|_______               
     |               VP            
     |        _______|___           
     |       |           PP        
     |       |    _______|___       
     NP      |   |           NP    
  ___|___    |   |        ___|___   
Det      N   V   P      Det      N 
 |       |   |   |       |       |  
the     cat sat  on     the     mat



In [9]:
print("PDF Sentence Parsing:")
for tree in parser.parse(pptx_sent_tokens):
    print(tree)
    tree.pretty_print()

PDF Sentence Parsing:
(S (NP (Det the) (N cat)) (VP (V chased) (NP (Det the) (N mouse))))
              S                 
      ________|_____             
     |              VP          
     |         _____|___         
     NP       |         NP      
  ___|___     |      ___|____    
Det      N    V    Det       N  
 |       |    |     |        |   
the     cat chased the     mouse



## Step 2: Create a 2D table to represent substrings of the input sentence.

In [10]:
import pandas as pd

In [11]:
def create_2D_table(sentence):
  '''
  create_2D_table is a function that


    1. Split the sentence into words
    2. Initialize an empty table
    3. Fill the table with substrings
    4. Creates a pandas DataFrame for better visualization

  input:
    a sentence
  output:
    a pandas dataframe of the 2D table representing the substrings
  '''

  words = sentence.split()
  table = [['' for _ in range(len(words))] for _ in range(len(words))]
  for i in range(len(words)):
    for j in range(i, len(words)):
      table[i][j] = ' '.join(words[i:j+1])

  out = pd.DataFrame(table, index=[f"Start {i}" for i in range(len(words))], columns=[f"End {i}" for i in range(len(words))])
  return out

In [12]:
# create the tables
substrtbl_pdf_sent = create_2D_table(pdf_sent)
substrtbl_pptx_sent = create_2D_table(pptx_sent)

In [13]:
# display the tables
substrtbl_pdf_sent

,End 0,End 1,End 2,End 3,End 4,End 5
Start 0,The,The cat,The cat sat,The cat sat on,The cat sat on the,The cat sat on the mat
Start 1,,cat,cat sat,cat sat on,cat sat on the,cat sat on the mat
Start 2,,,sat,sat on,sat on the,sat on the mat
Start 3,,,,on,on the,on the mat
Start 4,,,,,the,the mat
Start 5,,,,,,mat


In [14]:
substrtbl_pptx_sent

,End 0,End 1,End 2,End 3,End 4
Start 0,The,The cat,The cat chased,The cat chased the,The cat chased the mouse
Start 1,,cat,cat chased,cat chased the,cat chased the mouse
Start 2,,,chased,chased the,chased the mouse
Start 3,,,,the,the mouse
Start 4,,,,,mouse


## Step 3: Iterate over substrings and fill in the table based on grammar rules. Return whether the input sentence is valid or not according to the CFG

In [15]:
def cyk_truth_val(sentence, grammar):
  '''
  cyk_truth_val

  input:
    a sentence and its grammar
  output:
    a truth table and a truth value for if the imput sentence is valid or not
  '''
  word = sentence.split()
  n = len(word)

  table = [[set() for _ in range(n)] for _ in range(n)]
  truth_table = [[False for _ in range(n)] for _ in range(n)]

  productions = {}
  for production in grammar.productions():
    rhs = tuple(production.rhs())
    lhs = production.lhs()
    if rhs in productions:
      productions[rhs].append(lhs)
    else:
      productions[rhs] = [lhs]

  for i, word in enumerate(word):
    for rhs, lhs_list in productions.items():
      if len(rhs) == 1 and rhs[0] == word:
        table[i][i].update(lhs_list)
        truth_table[i][i] = True

  for length in range(2, n+1):
    for i in range(n-length+1):
      j = i + length - 1
      for k in range(i, j):
        for B in table[i][k]:
          for C in table[k+1][j]:
            if (B, C) in productions:
              table[i][j].update(productions[(B, C)])
              truth_table[i][j] = True
  valid = 'S' in table[0][n-1]
  return valid, pd.DataFrame(truth_table)

In [16]:
def isValid(truth, sent_name, parser):
  if truth:
    print("The sentence %s is a valid according to %s" %(sent_name, parser))
  print("The sentence %s is a valid according to %s" %(sent_name, parser))

In [17]:
pdf_valid, pdf_cyk_tab = cyk_truth_val(pdf_sent, grammar)
pptx_valid, pptx_cyk_tab = cyk_truth_val(pptx_sent, grammar)

### Process

1. The table is `filled in a bottom-up manner` starting with individual words and building up to larger substrings.
2. **For substrings of length 1** -- fill in the possible non-terminals that can generate the word (like NP -> Det N).
3. **For substrings of length > 1** -- the function splits the string into two parts and check if there are any grammar rules that can generate the combined substring.

In [18]:
# print the truth value
isValid(pdf_valid, "pdf sentence", "CFG")
pdf_cyk_tab

The sentence pdf sentence is a valid according to CFG


,0,1,2,3,4,5
0,False,False,False,False,False,False
1,False,True,False,False,False,False
2,False,False,True,False,False,True
3,False,False,False,True,False,True
4,False,False,False,False,True,True
5,False,False,False,False,False,True


In [19]:
isValid(pptx_valid, "power point sentence", "CFG")
pptx_cyk_tab

The sentence power point sentence is a valid according to CFG


,0,1,2,3,4
0,False,False,False,False,False
1,False,True,False,False,False
2,False,False,True,False,True
3,False,False,False,True,True
4,False,False,False,False,True


## Task 2: Use the following pseudocode structure to implement Earley’s algorithm:

1. Initialize the Earley chart for storing parsing states.
2. Iterate over input tokens and apply the predict, scan, and complete operations.
3. Parse the input sentence and check if it matches the grammar.


**Earley's Algorithm**

Earley's algorithm is an efficient parsing algorithm used to recognize strings in a context-free language and is particularly well-suited for parsing CFGs

**Advantage**

1. It is more general than other parsers like CYK because it can handle any CFG

2. Top-Down Parsing: It works in a top-down manner, which makes it easy to implement and understand compared to bottom-up algorithms like CYK.

**O notation and Efficiency**

- Earley’s algorithm operates in $O(n ^3)$ time in the worst case
- In unambiguous grammar it runs at $O(n^2)$
- It can run at $O(n)$

**Concepts**

1. **States** -- represents a point in a parse rule, showing what has been matched so far and what remains to be matched.

> A state is represented as $(X → α ⋅ β, i)$, where:
>
> $X → α β$ is a production rule.
>
> $\cdot$ -->  marks how much of the rule has been recognized.
>
> $i$ is the position in the input where this state started.
>
> For example, $(NP → Det ⋅ N, 0)$ means that we are trying to match the rule $NP → Det\ N$
>
> - we have already seen `a Det` starting from position `0`
>
> - but, we still need to match the `N`.

2. **Chart** -- a list of sets of states kept by the parser.

> **Components**
>
> a. one set for each position in the input
>
> b. an additional set at the end
>

3. **Operations** --

> a. **Prediction** - It adds new states for each possible rule for `A`. This means that if `A` is the next step to be matched, the algorithm will predict all the grammar rules that could produce `A`.
>
> b. **Scanning** - matches terminal symbols.  If the next symbol to be matched is a terminal and the current symbol in the input matches it, the parser advances the state by shifting the $⋅$ to the right.
>
> c. **Comoletion** - completes the rule. If a state has finished recognizing the right-hand side `. is at the end`. the parser looks back to previous states that were expecting this non-terminal and advances them.
>

In [20]:
# current state of the parser
class State:
  '''
  has information on:
    1. Left-hand side of the rule
    2. Right-hand side of the rule
    3. Position of the dot in RHS
    4. The position where the state begins
  '''
  def __init__(self, lhs, rhs, dot_ind, start_ind):
    self.lhs = lhs
    self.rhs = rhs
    self.dot_ind = dot_ind
    self.start_ind = start_ind

  def __repr__(self):
    '''
    how much of the sentence-building puzzle you've completed so far.
    '''
    lhs_str = str(self.lhs)
    rhs_before_dot = ' '.join([str(symbol) for symbol in self.rhs[:self.dot_ind]])
    rhs_after_dot = ' '.join([str(symbol) for symbol in self.rhs[self.dot_ind:]])
    return f"{lhs_str} -> {rhs_before_dot} • {rhs_after_dot}, {self.start_ind}"


  def is_complete(self):
    '''
    completion
    '''
    return self.dot_ind == len(self.rhs)

  def next_symbol(self):
    '''
    scanning
    '''
    if not self.is_complete():
      return self.rhs[self.dot_ind]
    return None

  def advance(self):
    '''
    refresh the left and right
    move the dot to the next position
    update the index to start from
    '''
    return State(self.lhs, self.rhs, self.dot_ind + 1, self.start_ind)


In [21]:
# Earley parser
class EarleyParser:
  def __init__(self, grammar):
    self.grammar = grammar
    self.chart = [] # set of the positions in the input
    self.non_terminals = set(prod.lhs() for prod in grammar.productions())  # Set of non-terminals


  def init_chart(self, sent_len):
    # create a chart with one more step after the end
    self.chart = [[] for _ in range(sent_len + 1)]

  def add_state(self, state, index):
    '''
    populate the chart
    '''
    if state not in self.chart[index]:
      self.chart[index].append(state)

  def predict(self, state, index):
    '''
    new states for each possible rule of A
    '''
    # inheit the next_symbol
    next_symbol = state.next_symbol()
    # ther is a next symbol and the next symbol is not a non terminal
    if next_symbol is not None and next_symbol in self.non_terminals:
      for prod in self.grammar.productions(lhs = next_symbol):
        new_state = State(prod.lhs(), prod.rhs(), 0, index)
        self.add_state(new_state, index)

  def scan(self, state, word, index):
    '''
    shift dot to the right
    '''
    next_symbol = state.next_symbol()
    if next_symbol == word:
      new_state = state.advance()
      self.add_state(new_state, index + 1)

  def complete(self, state, index):
    if state.is_complete():
      for prev_state in self.chart[state.start_ind]:
        if prev_state.next_symbol() == state.lhs:
          new_state = prev_state.advance()
          self.add_state(new_state, index)

  def parse(self, sentence):
    '''
    perform the earley's parsing
    '''
    words = sentence.split()
    self.init_chart(len(words))

    start_rule = self.grammar.productions(lhs=self.grammar.start())[0]
    start_state = State(start_rule.lhs(), start_rule.rhs(), 0, 0)
    self.add_state(start_state, 0)

    for i in range(len(words) + 1):
      for state in self.chart[i]:
        if not state.is_complete():
          if state.next_symbol() in self.non_terminals:
            self.predict(state, i)
          elif i < len(words):
            self.scan(state, words[i], i)
        else:
          self.complete(state, i)

    self.print_chart(words)
    # check complete
    for state in self.chart[len(words)]:
      if state.lhs == self.grammar.start() and state.is_complete() and state.start_index == 0:
        return True
    return False

  def print_chart(self, words):
    '''
    Print the Earley chart.
    '''
    print("\nEarley Chart:")
    for i, states in enumerate(self.chart):
      word = words[i-1] if i > 0 else "Start"
      print(f"Chart[{i}] (after '{word}'):")
      for state in states:
        completed_percentage = round((state.dot_ind / len(state.rhs)) * 100) if len(state.rhs) > 0 else 100
        print(f"  {state} (Progress: {completed_percentage}%)")
    print()


In [22]:
earley = EarleyParser(grammar)
earley_pdf = earley.parse(pdf_sent)



Earley Chart:
Chart[0] (after 'Start'):
  S ->  • NP VP, 0 (Progress: 0%)
  NP ->  • Det N, 0 (Progress: 0%)
  Det ->  • the, 0 (Progress: 0%)
Chart[1] (after 'The'):
Chart[2] (after 'cat'):
Chart[3] (after 'sat'):
Chart[4] (after 'on'):
Chart[5] (after 'the'):
Chart[6] (after 'mat'):



In [23]:
earley_pptx = earley.parse(pptx_sent)


Earley Chart:
Chart[0] (after 'Start'):
  S ->  • NP VP, 0 (Progress: 0%)
  NP ->  • Det N, 0 (Progress: 0%)
  Det ->  • the, 0 (Progress: 0%)
Chart[1] (after 'The'):
Chart[2] (after 'cat'):
Chart[3] (after 'chased'):
Chart[4] (after 'the'):
Chart[5] (after 'mouse'):



In [24]:
isValid(earley_pdf, "pdf sentence", "Earley")

The sentence pdf sentence is a valid according to Earley


In [25]:
isValid(earley_pptx, "power point sentence", "Earley")

The sentence power point sentence is a valid according to Earley


## Task 3:

Try parsing complex sentences or experiment with different CFGs.
Compare the performance and efficiency of CKY vs. Earley when handling ambiguous or complex sentences.



In [26]:
complex_grammar = CFG.fromstring("""
    S -> NP VP
    S -> S Conj S
    NP -> Det N | Det Adj N | N
    VP -> V NP | V PP | V NP PP | V
    PP -> P NP
    Adj -> 'small' | 'large' | 'red'
    N -> 'cat' | 'dog' | 'mouse' | 'garden'
    V -> 'chased' | 'saw' | 'ran'
    P -> 'in' | 'near'
    Det -> 'the' | 'a'
    Conj -> 'and' | 'or'
""")

complex_sentence = "the small cat ran near the large dog and chased a red small mouse in the garden"

In [27]:
def print_cky_progress(table, words):
  """
  Print the CKY table and show how much of the sentence has been generated at each step.
  """
  print("\nCKY Table Progress:")
  n = len(words)
  for length in range(1, n+1):
    for i in range(n-length+1):
      j = i + length - 1
      print(f"Substring ({i}, {j}) - '{' '.join(words[i:j+1])}':")
      if len(table[i][j]) > 0:
        print(f"  Non-terminals: {table[i][j]} (Progress: 100%)")
      else:
        print(f"  Non-terminals: None (Progress: 0%)")


### Test and compare

there is an earley chart parser package in `nltk.parse.earleychart`

it seems to be optimezed for earley parsing and saves on memory. the comparison shall be use this to try and save on available memory.

In [28]:
from nltk import Nonterminal, ChartParser
from nltk.parse.earleychart import EarleyChartParser
import time

In [29]:
class CFGParserComparison:
    def __init__(self, grammar):
        self.grammar = grammar
        self.cky_parser = ChartParser(self.grammar)

    def cky_parse(self, sentence):
        tokens = sentence.split()
        start_time = time.time()
        parses = list(self.cky_parser.parse(tokens))
        end_time = time.time()
        return parses, end_time - start_time

    def earley_parse(self, sentence):
        tokens = sentence.split()
        earley_parser = EarleyChartParser(self.grammar)
        start_time = time.time()
        parses = list(earley_parser.parse(tokens))
        end_time = time.time()
        return parses, end_time - start_time

    def compare_parsers(self, sentence):
        print(f"Parsing sentence: '{sentence}'\n")

        # CKY Parse
        cky_parses, cky_time = self.cky_parse(sentence)
        print("CKY Parser:")
        if cky_parses:
            for tree in cky_parses:
                print(tree)
                tree.pretty_print()
        else:
            print("No parse found.")
        print(f"Time taken: {cky_time:.6f} seconds\n")

        # Earley Parse
        earley_parses, earley_time = self.earley_parse(sentence)
        print("Earley Parser:")
        if earley_parses:
            for tree in earley_parses:
                print(tree)
                tree.pretty_print()
        else:
            print("No parse found.")
        print(f"Time taken: {earley_time:.6f} seconds\n")

In [30]:
compare_parsing = CFGParserComparison(complex_grammar)
compare_parsing.compare_parsers(complex_sentence)

Parsing sentence: 'the small cat ran near the large dog and chased a red small mouse in the garden'

CKY Parser:
No parse found.
Time taken: 0.008949 seconds

Earley Parser:
No parse found.
Time taken: 0.003717 seconds



In [32]:
combined_sent = ['small mouse', 'the dog in a garden', 'cat and mouse']
for sentence in combined_sent:
  compare_parsing = CFGParserComparison(complex_grammar)
  compare_parsing.compare_parsers(sentence)


Parsing sentence: 'small mouse'

CKY Parser:
No parse found.
Time taken: 0.000447 seconds

Earley Parser:
No parse found.
Time taken: 0.000453 seconds

Parsing sentence: 'the dog in a garden'

CKY Parser:
No parse found.
Time taken: 0.000870 seconds

Earley Parser:
No parse found.
Time taken: 0.001195 seconds

Parsing sentence: 'cat and mouse'

CKY Parser:
No parse found.
Time taken: 0.000591 seconds

Earley Parser:
No parse found.
Time taken: 0.001777 seconds



## Task 4: Provide a lab report documenting the results and performance of the parsing algorithms. (Include this at the bottom of the notebook)

Explain the output for multiple test sentences parsed using CKY and Earley.
Discuss the limitations of each algorithm and where one might be preferred over the other.